### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re

#import pymysql
import getpass
#from sshtunnel import SSHTunnelForwarder

from IPython.display import clear_output

import csv
import xlrd
import xlwt

import getpass
from sshtunnel import SSHTunnelForwarder
import pymysql

import unicodedata
import numpy as np
from collections import OrderedDict, Counter
import nltk
from nltk.collocations import *
from nltk.util import ngrams
import string
import pickle

Especifica qual sistema operacional está sendo usado. Renato = Linux ; Marcelo = nt (Windows) e define variáveis a partir disso

In [2]:
if os.name == 'nt':
    ssh_priv_key = 'C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa'
    ssh_user='marcelobribeiro'
    sql_user='marcelobribeiro'
    path_inputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    #path_inputs = "C:/Users/MARCELO/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"
    path_outputs = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/outputs/"
else:
    ssh_priv_key = '/home/rsouza/.ssh/id_rsa'
    ssh_user='rsouza'
    sql_user='rsouza'
    path_inputs = "/home/rsouza/Documentos/text-learning-tools/inputs/"
    path_outputs = "/home/rsouza/Documentos/text-learning-tools/outputs/"

### Captura lista de nomes da base Accessus/CPDOC

Abre arquivo XLS

In [11]:
workbook = xlrd.open_workbook(path_inputs+'PESSOAS-do-Accessus.xls', on_demand = True)

#worksheet = workbook.sheet_by_index(0)
worksheet = workbook.sheet_by_name('Rel_DescritorEleito')

WARNING *** file size (1367936) not 512 + multiple of sector size (512)


In [12]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

Retira termos obtidos da lista de nomes, obtida em teste preliminar, que não são pessoas.

In [13]:
not_persons_list = ['manifestações', 'manobras', 'mão de obra', 'marxismo', 'materialismo', 'medicina', 'medidas', 'meeiros', 'meio', 'mercado', 
 'mercantilismo', 'messianismo', 'metrologia', 'ministério', 'missão', 'mobilidade', 'mobilização', 'modelo', 'monarquia', 
 'monopólio', 'mortalidade', 'movimento', 'mudança', 'municípios', 'nação', 'nacionalismo', 'negros']

In [14]:
cpdoc_names_list = {}

for line in range(worksheet.nrows):
    breakpoint = 0
    var_names = []
    number_name = str(worksheet.cell(line, 0).value)
    check_number = re.search('^\d',number_name)
    if check_number is not None:
        if line == worksheet.nrows-1: break
        fullname = str(worksheet.cell(line, 1).value)
        fullname = str.lower(fullname)
        for not_person in not_persons_list:
            if not_person in fullname: breakpoint = 1
        if breakpoint == 1: continue
        fullname_strip_accent = strip_accents(fullname)
        fullname_strip_abrev = re.sub('(\w\. ){0,4}', r'', fullname)
        fullname_strip_both = strip_accents(fullname_strip_abrev)
        var_names.append(fullname)
        if fullname_strip_accent != fullname: var_names.append(fullname_strip_accent)
        if fullname_strip_abrev  != fullname: var_names.append(fullname_strip_abrev)
        if fullname_strip_both   != fullname_strip_abrev and fullname_strip_both != fullname_strip_accent: 
            var_names.append(fullname_strip_both)
        cpdoc_names_list[fullname] = var_names

In [56]:
sorted(cpdoc_names_list)[100:110]

['adelmo lodi',
 'ademar alegria',
 'ademar braga',
 'ademar de barros',
 'ademar de barros filho',
 'ademar de faria',
 'ademar de melo ',
 'ademar de queiroz',
 'ademar galvão',
 'ademar gutierrez']

### Captura lista de nomes da base History-Lab.

In [91]:
count = 0
texts = []
frus_names_list = {}

#pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
conn = pymysql.connect(host='history-lab.org', 
                       user='de_reader',
                       passwd=pass_mysql,
                       db='declassification_frus',
                       use_unicode=True, 
                       charset="utf8")
cur = conn.cursor()
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
insere dados na base mysql
captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

cur.execute("SELECT id, name FROM persons")
data = cur.fetchall()

for row in data:
    if row is None: break
    var_names = []
    count += 1 # for break event
    fullname =  row[1]
    fullname = str.lower(fullname)
    fullname = re.sub('(.*), (.*)', r'\2 \1', fullname)
    fullname = re.sub('(.*)\(.*\)\s?(.*)', r'\1\2', fullname)
    fullname = re.sub('- ', '-', fullname)
    fullname_strip_accent = strip_accents(fullname)
    fullname_strip_abrev = re.sub('(\w\. ){0,4}', r'', fullname)
    fullname_strip_both = strip_accents(fullname_strip_abrev)
    var_names.append(fullname)
    if fullname_strip_accent != fullname: var_names.append(fullname_strip_accent)
    if fullname_strip_abrev  != fullname: var_names.append(fullname_strip_abrev)
    if fullname_strip_both   != fullname_strip_abrev and fullname_strip_both != fullname_strip_accent: 
        var_names.append(fullname_strip_both)
    frus_names_list[fullname] = var_names    
    #if count == 10: break #amostra de apenas x linhas

········


In [93]:
len(frus_names_list)

16284

### Cria lista de nomes duplicados e remove das outras listas

In [24]:
duplicate_names_list = {}
for name, values in cpdoc_names_list.items():
    for alt_name in values:
        if alt_name in frus_names_list:
            duplicate_names_list[name] = cpdoc_names_list[name]

Remove das outras listas

In [26]:
for name in duplicate_names_list:
    cpdoc_names_list.pop(name, None)
    frus_names_list.pop(name, None)   

### Captura textos da base CPDOC_AS do mysql.

In [19]:
count = 0
texts = ''

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    
    for row in data:
        if row is None: break
        count += 1 # for break event
        text =  row[1]
        texts += '\r\n'+ text
        #if count == 10: break #amostra de apenas x linhas

········
········


### Faz mineração de dados: checa a existência de cada nome das listas em todo o corpus. Cria novas listas com os nomes checados.
OBS: Essa primeira checagem captura o máximo possível de nomes, pois busca tanto por nomes quanto sobrenomes. Será necessário fazer uma segunda checagem para verificar se o nome exato ocorre, para evitar falsos positivos desnecessários e também para evitar ambiguidades.

In [ ]:
(cpdoc_names_check, frus_names_check, duplicate_names_check) = {}, {}, {}
collections_names_check = [cpdoc_names_check, frus_names_check, duplicate_names_check]
collections_names_list = [cpdoc_names_list, frus_names_list, duplicate_names_list]
for names_list in collections_names_list:
    count = 0
    percentil = int(len(names_list)/100)
    collections_names_list_index = collections_names_list.index(names_list)
    names_check = collections_names_check[collections_names_list_index]
    for name,alt_names in names_list.items():
        
        count += 1
        if count % percentil == 0: 
                clear_output()
                print(int(count/percentil),'% done on list',collections_names_list.index(names_list))
        
        for i in alt_names:
            if i in texts:
                names_check[name] =  names_list[name]
                break
    

In [38]:
print(len(cpdoc_names_check))
print(len(frus_names_check))
print(len(duplicate_names_check))
#sorted(duplicate_names_check)[:10]

1022
922
66


Cria lista de nomes que aparecem no corpus, mas estão fora das listas do accessus e do history-lab, ma. Nota: alguns nomes não estão identificados.

In [54]:
azeredopapers_names_check = {}
new_names = ['debernardi', 'alberto nogués', 'hans-dietrich genscher', 'allara', 'akasaka', 'sauvagnargues', 'andrei gromyko']
for name in new_names: azeredopapers_names_check[name] = [name]

In [ ]:
for name in new_names: cpdoc_names_check.pop(name, None)    

### Salva as 3 listas de nomes criadas (ou as recarrega)

In [4]:
cpdoc_names_check_file = path_outputs+'/corpus_entities_persons_list_cpdoc.pkl'
frus_names_check_file = path_outputs+'/corpus_entities_persons_list_frus.pkl'
duplicate_names_check_file = path_outputs+'/corpus_entities_persons_list_duplicate.pkl'
azeredopapers_names_check_file = path_outputs+'/corpus_entities_persons_list_azeredopapers.pkl'

In [5]:
''' caso queira carregar os arquivos '''
cpdoc_names_check = pickle.load(open(cpdoc_names_check_file, 'rb'))
frus_names_check = pickle.load(open(frus_names_check_file, 'rb'))
duplicate_names_check = pickle.load(open(duplicate_names_check_file, 'rb'))
azeredopapers_names_check = pickle.load(open(azeredopapers_names_check_file, 'rb'))

### Faz segunda checagem de nomes (duplicidades e ambiguidades), agora com a lista de nomes presentes no corpus

Retira nomes ambíguos por não serem nomes compostos (sem sobrenome em geral), preservando exceções.

In [6]:
ambiguous_list = []
exception_names = ['hirohito', 'yoshihito', 'akihito']
for fullname in frus_names_check:
    if " " not in fullname and fullname not in exception_names:
        ambiguous_list.append(fullname)
for fullname in ambiguous_list:
    frus_names_check.pop(fullname, None)
for fullname in frus_names_check:
    if " " not in fullname:
        print(fullname)

akihito
hirohito
yoshihito


Retira mais termos que não são pessoas

In [7]:
non_names = ['moçambique', 'microfilmagem', 'marrocos', 'méxico', 'camboja', 'maçonaria', 'allara', 'niterói', 'nicarágua', 'misticismo', 
 'militarismo', 'legislação', 'burundi', 'música', 'maranhão', 'nova república', 'america do norte', 'mar territorial']
for i in non_names:
    cpdoc_names_check.pop(i, None)    

Lista nomes que se tornam ambíguos ao retirar as abreviações.

In [8]:
for fullname in cpdoc_names_check:
    fullname_strip_abrev = re.sub('(\w\. ){0,4}', r'', fullname)
    if " " not in fullname_strip_abrev and fullname_strip_abrev != fullname:
        print(fullname)

e. l. resende
j. peres
k. kato
i. c. tupper
h. s. owen
a. faria
j. h. russo
j. e. brant
b. goulart
m. france
m. bastos
d. a. doran
e. m. feinberg
n. brandão
c. h. adams
d. gregory
j. a. wright
c. k. yen
j. j. seabra
a. azevedo
f. chermont
y. k. pao


Corrige nomes ambíguos, escritos de outra forma.

In [9]:
#nomes duplicados 
duplicate_names_check['hirohito'] = frus_names_check['hirohito']
cpdoc_names_check.pop('imperador hirohito', None) 
frus_names_check.pop('hirohito', None) 

#ocorrências incorretas
#cpdoc_names_check.pop('albert probst', None)

['hirohito']

Verifica frequência de sobrenomes (retira primeiro nome e abreviações)

In [10]:
surnames_list = []
for fullname in cpdoc_names_check:
    fullname = re.sub('(\w\. ){0,4}', r'', fullname)
    fullname = re.sub('( d[aeo] )', r' ', fullname)
    fullname = re.sub('\w+ (.*)', r'\1', fullname)
    fullname = fullname.split(' ')
    for name in fullname:
        surnames_list.append(name)
for fullname in frus_names_check:
    fullname = re.sub('(\w\. ){0,4}', r'', fullname)
    fullname = re.sub('( d[aeo] )', r' ', fullname)
    fullname = re.sub('\w+ (.*)', r'\1', fullname)
    fullname = fullname.split(' ')
    for name in fullname:
        surnames_list.append(name)

In [11]:
surnames_list[:10]

['augusto',
 'monte',
 'lins',
 'soares',
 'resende',
 'falcão',
 'johnston',
 'lira',
 'pereira',
 'leite']

Checa a existência de nomes com sobrenomes ambíguos

In [12]:
#sorted(cpdoc_names_check)

### Organizando nomes (do cpdoc) em tabela

In [13]:
#name_prepositions_list = ['de', 'do'] # obsoleto, mas pode ser útil para outras funções

cpdoc_series = pd.Series(cpdoc_names_check)
cpdoc_data = pd.DataFrame(cpdoc_series, columns=['alt_names'])
cpdoc_data.index.name = 'names'
cpdoc_data['has_ambiguous_surname'], cpdoc_data['short_fullname'], cpdoc_data['short_surname'] = 'False', 'False', 'False'
for row in cpdoc_data.itertuples():
    
    # check if fullname is too short
    for alt_name in row[1]: 
        alt_name = re.sub('^(d[aeo]s? )', '', alt_name)
        alt_name = re.sub('( d[aeo]s? )', ' ', alt_name)
        if " " not in alt_name: cpdoc_data.ix[row[0], 'short_fullname'] = 'True'
               
    # check if fullname has a too ambiguous surname, or if fullname has a too short surname
    fullname = row[0]
    fullname = re.sub('\w+ (.*)', r'\1', fullname)
    fullname = re.sub('(\w\. ){0,4}', '', fullname)
    fullname = re.sub('^(d[aeo]s? )', '', fullname)
    fullname = re.sub('( d[aeo]s? )', ' ', fullname)
    fullname = fullname.split(' ')
    for name in fullname:
        count = 0
        for key in cpdoc_names_check.keys():
            if name in key:
                count += 1
        if count > 1: cpdoc_data.ix[row[0], 'has_ambiguous_surname'] = 'True'         
        if len(name) < 5: cpdoc_data.ix[row[0], 'short_surname'] = 'True'

    #if row[0] == 'abreu lima': break

In [14]:
cpdoc_data[0:20]

,alt_names,has_ambiguous_surname,short_fullname,short_surname
names,,,,
a. azevedo,"[a. azevedo, azevedo]",True,True,False
a. faria,"[a. faria, faria]",True,True,False
a. i. de lima,"[a. i. de lima, de lima]",True,True,True
a. j. peixoto de castro,"[a. j. peixoto de castro, peixoto de castro]",True,False,False
abgar renault,[abgar renault],False,False,False
abreu lima,[abreu lima],True,False,True
adalberto de barros nunes,[adalberto de barros nunes],True,False,False
adalberto pereira dos santos,[adalberto pereira dos santos],True,False,False
adalberto sena,[adalberto sena],False,False,True


### Inserção na base de dados

In [15]:
count = 0

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key=ssh_priv_key,
                        ssh_private_key_password=pass_sshkey,
                        ssh_username=ssh_user,
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user=sql_user,
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

    cur.execute("DROP TABLE IF EXISTS persons")
    cur.execute('''CREATE TABLE IF NOT EXISTS persons
               (id VARCHAR(128) PRIMARY KEY, person_name VARCHAR(128)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
    cur.execute("DROP TABLE IF EXISTS person_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS person_doc
               (person_id VARCHAR(128), doc_id VARCHAR(31), person_count SMALLINT(5)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')
    
    cur.execute("SELECT * FROM CPDOC_AS.docs WHERE main_language = 'pt' AND (readability > 0.4 OR readability = -1) ")  
    
    text_data = cur.fetchall()
    
    numrows_df = len(cpdoc_data.index)
    percentil = int(numrows_df/100)
    
    for row_name in cpdoc_data.itertuples():
        person_id = count + 500000 # id do History-Lab termina em 116.000. Precisamos ter uma sequência de ID distante, tal como foi acordado entre Marcelo e Rohan
        name = row_name[0]
        alt_names = cpdoc_data.ix[row_name[0], 'alt_names']
        
        ### mede percentual de conclusão da tarefa ###
        if count % 100 == 0: 
            clear_output()
            print(int((count)/percentil),'% done')
        
        for row_text in text_data:
            text = row_text[4]
            doc_id = row_text[0]
                        
            person_count = 0
            if cpdoc_data.ix[row_name[0], 'has_ambiguous_surname'] == 'True' or cpdoc_data.ix[row_name[0], 'short_surname'] == 'True' or cpdoc_data.ix[row_name[0], 'short_fullname'] == 'True': 
                person_count = text.count(name)
            else: 
                for name in alt_names:
                    person_subfreq = text.count(name)
                    person_count += person_subfreq
            if person_count > 0: 
                query = "INSERT INTO person_doc VALUES (%s, %s, %s)"
                cur.execute(query, (person_id, doc_id, person_count))
        
        query = "INSERT INTO persons VALUES (%s, %s)"
        cur.execute(query, (person_id, name))
        
        count += 1
        #if count >= 100: break #amostra de apenas x linhas ~ 10%
    
    # acrescenta colunas presentes na base do History-Lab (default = nulo)
    cur.execute("ALTER TABLE persons ADD birth_year INT(4)")
    cur.execute("ALTER TABLE persons ADD death_year INT(4)")
    cur.execute("ALTER TABLE persons ADD description MEDIUMTEXT")
    cur.execute("ALTER TABLE person_doc ADD date DATETIME")

100 % done


### Análise Exploratória de Dados

In [35]:
#query = 'azeredo'
count = 0
for name in cpdoc_names_check:
    count += 1
    query = name
    tknzr = nltk.tokenize.WordPunctTokenizer()
    tokens = tknzr.tokenize(texts) # texto com stopwords
    #tokens = tknzr.tokenize(cleaned_texts) # texto sem stopwords
    #print(tokens)
    try:
        print(tokens[tokens.index(name)-40 : tokens.index(name)+40])
        tokens = tokens[tokens.index(name)+1:]
        print(tokens[tokens.index(name)-40 : tokens.index(name)+40])
    except: continue
    if count == 50: break
    #nltk_text = nltk.Text(tokens)
    #nltk_text.concordance(query.lower(), width=60, lines=10,)

['own', 'party', 'and', 'splitting', 'it', 'in', 'the', 'process', ';', 'mirabeau', 'prvsiding', 'over', 'an', 'attempt', 'at', 'legal', 'revolution', '.', 'bismarck', 'was', 'back', 'on', 'the', 'ancestral', 'estate', 'now', '.', 'restless', ',', 'he', 'read', 'voraciously', ':', 'shakespeare', 'and', 'byron', ',', 'louis', 'blanc', 'and', 'voltaire', ',', 'and', 'always', 'spinoza', '.', 'his', 'escapades', 'multiplied', '.', 'after', 'another', 'hrokcn', 'engagement', ',', 'bismarck', 'lcft', 'on', '&', 'journey', 'through', 'england', ',', 'france', ',', 'and', 'switzerland', '.', 'he', 'even', 'made', 'tentative', 'plans', 'to', 'serve', 'with', 'the', 'british', 'army', 'in']
['represas', '.', 'desmentido', ',', 'como', 'vimos', ',', 'el', 'mismo', '-', 'dia', 'sicuiente', '.', 'el', 'optimismo', 'es', 'a', 'veces', 'un', 'fenomeno', 'muy', 'curioso', '.', ';--', 'el', 'patrono', 'literario', 'de', 'todos', 'los', 'optimistas', 'es', 'aquel', 'famoso', 'personaje', 'del', '"', 'c

KeyboardInterrupt: 

In [38]:
type(nltk_text)

list

In [25]:
nltk_text.similar(query)

In [26]:
nltk_text.collocations(num=20, window_size=2)

agradeço prezado; das relações; marcio muller; new bedford; relações
exteriores; brazilian consulate; muller bueno; professor haroldo;
souto 408; vieira souto; afetuoso abraço; amigo generosas; prezado
amigo; para embaixador; embaixador azeredo; azeredo silveira


Collocations are expressions of multiple words which commonly co-occur.
Below we are using Pointwise Mutual Information.
http://en.wikipedia.org/wiki/Pointwise_mutual_information

In [31]:
max_items = 50
freq_min_b = 2
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder2 = nltk.collocations.BigramCollocationFinder.from_words(nltk_text)
finder2.apply_word_filter(lambda w: w in string.punctuation)
finder2.apply_freq_filter(freq_min_b)
for a, b in finder2.nbest(bigram_measures.pmi, max_items):
    print (a, b)

agradeço prezado
das relações
marcio muller
new bedford
relações exteriores
brazilian consulate
muller bueno
professor haroldo
b rjo
souto 408
vieira souto
afetuoso abraço
amigo generosas
do brasil
prezado amigo
telegrama no
para embaixador
pt afetuoso
azeredo da
embaixador azeredo
da silveira
rio gb
t e
azeredo silveira
amigo pt
de estado
de novembro
secretaria de
15 de
.. ..
em de
de de


In [32]:
max_items = 50
freq_min_t = 2
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder3 = nltk.collocations.TrigramCollocationFinder.from_words(nltk_text)
finder3.apply_word_filter(lambda w: w in string.punctuation)
finder3.apply_ngram_filter(lambda w1, w2, w3:  w1 in ['da', 'de', 'das'])
finder3.apply_freq_filter(freq_min_t)
for a, b, c in finder3.nbest(trigram_measures.pmi, max_items):
    print (a, b, c)

marcio muller bueno
agradeço prezado amigo
prezado amigo generosas
pt afetuoso abraço
secretaria de estado
15 de novembro
azeredo da silveira
amigo pt afetuoso
.. .. ..


In [33]:
for a, b in (finder2.above_score(bigram_measures.raw_freq,1.0 / len(list(nltk.bigrams(tokens))))):
        print (a, b)

.. ..
pt afetuoso
rio gb
da silveira
15 de
afetuoso abraço
agradeço prezado
amigo generosas
amigo pt
azeredo da
azeredo silveira
b rjo
brazilian consulate
das relações
de de
de estado
de novembro
do brasil
em de
embaixador azeredo
marcio muller
muller bueno
new bedford
para embaixador
prezado amigo
professor haroldo
relações exteriores
secretaria de
souto 408
t e
telegrama no
vieira souto


In [34]:
for a, b, c in finder3.above_score(trigram_measures.raw_freq,1.0 / len(list(nltk.trigrams(tokens)))):
        print (a, b, c)

.. .. ..
15 de novembro
agradeço prezado amigo
amigo pt afetuoso
azeredo da silveira
marcio muller bueno
prezado amigo generosas
pt afetuoso abraço
secretaria de estado


In [35]:
n = 4
xgrams = ngrams(tokens, n)
xgrams_counter = Counter(xgrams)
series_xgrams_counter = pd.Series(data=xgrams_counter, index=xgrams_counter.keys())
df_ngrams = pd.DataFrame(series_xgrams_counter, columns = ['Frequência'])
df_ngrams = df_ngrams.sort_values(by='Frequência', ascending=False)
df_ngrams.index.name = 'Tokens'
df_ngrams[0:40]

,Frequência
Tokens,
"(.., .., .., ..)",13
"(., .., .., ..)",2
"(agradeço, prezado, amigo, generosas)",2
"(., .., ,, .)",2
"(r, a, m, a)",1
"(/, 1244, %, ãdãaam)",1
"(_, ., ., -)",1
"(mv, ,, ;, ()",1
"(e, horas, x, ""?)",1


# RASCUNHOS

In [146]:
cfd = nltk.ConditionalFreqDist(
    (target, fileid[:4])
    for fileid in inaugural.fileids()
    for w in inaugural.words(fileid)
    for target in ['america', 'citizen']
    if w.lower().startswith(target)) [1]
cfd.plot()

NameError: name 'inaugural' is not defined

##### ENTIDADE: PESSOA

Alimenta a tabela person_doc, da nossa base no mysql, fazendo interseção de dados da tabela persons com a tabela docs.

In [12]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
alimenta a a tabela person_doc, da nossa base no mysql.
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS person_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS person_doc 
               (person_id INT(11), doc_id VARCHAR(31), person_count BIGINT(21)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text_id = row[0]
        text =  row[1]
        #text = text.split()
        doc_id = row[0]
        for name in names_list_test:
            person_id = 1
            person_count = text.count(name)
            #query = "INSERT INTO person_doc VALUES (%s, %s, %s)"
            #cur.execute(query, (person_id, doc_id, person_count))
        print(name, 'text_id', text_id)
        #print(doc_id, topic_id, topic_score)   
        #cur.execute("ALTER TABLE person_doc ORDER BY doc_id ASC, person_count DESC")
        
        if count == 2: break #amostra de apenas 10 linhas

········
········
azeredo text_id rb_1974.04.17_doc_I-66
azeredo text_id onu_1974.03.12_doc_II-49
azeredo text_id onu_1974.03.12_doc_V-2
azeredo text_id be_1974.04.16_doc_VI-18
azeredo text_id be_1977.06.01_doc_II-21
azeredo text_id ag_1974.03.13_doc_III-7
azeredo text_id ag_1973.11.20_doc_VI-75
azeredo text_id d_1974.04.23_doc_XXXII-8
azeredo text_id d_1974.03.26_doc_XXXII-32
azeredo text_id d_1974.03.26_doc_XXII-9


##### ENTIDADE: LUGAR (PAÍS)

Alimenta a a tabela country_doc, da nossa base no mysql, fazendo interseção de dados da tabela countries com a tabela docs.